In [3]:
# ============================================================================
# Convert FITS Images to Float64 (Real) Format
# ============================================================================
# Converts uint16/int32/etc FITS images to float64 and saves as *_real.fits

import numpy as np
from astropy.io import fits
import glob
import os

# ============================================================================
# CONFIGURATION
# ============================================================================

INPUT_FOLDER = "../../../../../"  # Folder with FITS files
INPUT_PATTERN = "*.fits"  # Pattern to match (e.g., "*.fits", "science_*.fits")
OUTPUT_SUFFIX = "_real"  # Suffix to add before .fits (name_real.fits)

# ============================================================================
# CONVERSION
# ============================================================================

print("=" * 80)
print("Convert FITS Images to Float64 Format")
print("=" * 80)

# Find all FITS files in INPUT_FOLDER only (no subdirectories)
search_pattern = os.path.join(INPUT_FOLDER, INPUT_PATTERN)
fits_files = glob.glob(search_pattern)

print(f"Searching in: {INPUT_FOLDER}")
print(f"Pattern: {INPUT_PATTERN}")
print(f"Found {len(fits_files)} FITS files\n")

for i, filepath in enumerate(fits_files):
    print(f"[{i+1}/{len(fits_files)}] Processing: {os.path.basename(filepath)}")
    
    try:
        # Load FITS file
        with fits.open(filepath) as hdul:
            header = hdul[0].header
            data = hdul[0].data
            
            # Show original data type
            print(f"  Original dtype: {data.dtype}")
            print(f"  Shape: {data.shape}")
            print(f"  Range: [{np.min(data):.1f}, {np.max(data):.1f}]")
            
            # Convert to float64
            data_real = data.astype(np.float64)
            
            # Create output filename
            base_path = os.path.splitext(filepath)[0]  # Remove .fits
            if filepath.endswith('.fits.gz'):
                base_path = os.path.splitext(base_path)[0]  # Remove .gz too
            
            output_path = f"{base_path}{OUTPUT_SUFFIX}.fits"
            
            # Save as new FITS file
            hdu_out = fits.PrimaryHDU(data_real)
            
            # Copy all original header keywords
            hdu_out.header = header.copy()
            
            # Add conversion info to header
            hdu_out.header['ORIGTYPE'] = (str(data.dtype), 'Original data type')
            hdu_out.header['CONVERTED'] = (True, 'Converted to float64')
            hdu_out.header['BITPIX'] = -64  # FITS code for float64 (not 32!)
            
            hdu_out.writeto(output_path, overwrite=True)
            
            print(f"  Saved: {os.path.basename(output_path)}")
            print(f"  New dtype: {data_real.dtype}\n")
            
    except Exception as e:
        print(f"  ERROR: {e}\n")
        continue

print("=" * 80)
print(f"Done! Converted {len(fits_files)} files")
print(f"Output files saved with suffix: {OUTPUT_SUFFIX}.fits")

Convert FITS Images to Float64 Format
Searching in: ../../../../../
Pattern: *.fits
Found 0 FITS files

Done! Converted 0 files
Output files saved with suffix: _real.fits
